BERT로 텍스트 분류
https://www.tensorflow.org/text/tutorials/classify_text_with_bert?hl=ko

BERT 소개
BERT 및 기타 변압기 인코더 아키텍처는 NLP의 다양한 작업 (자연 언어 처리)에 격렬하게 성공했다. 그들은 딥 러닝 모델에 사용하기에 적합한 자연어의 벡터 공간 표현을 계산합니다. BERT 모델 제품군은 Transformer 인코더 아키텍처를 사용하여 이전 및 이후의 모든 토큰의 전체 컨텍스트에서 입력 텍스트의 각 토큰을 처리하므로 이름: Bidirectional Encoder Representations from Transformers.

BERT 모델은 일반적으로 대량의 텍스트에 대해 사전 학습된 다음 특정 작업에 맞게 미세 조정됩니다.

pip install -q -U tensorflow-text

In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

c:\Users\deepBlue\deepblue\compitition\dacon\daconenv\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\deepBlue\deepblue\compitition\dacon\daconenv\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounte

tf.get_logger().setLevel('ERROR')은 TensorFlow의 로깅 레벨을 설정하는 코드입니다. 
- 로깅은 프로그램 실행 중에 발생하는 메시지를 기록하고 출력하는 작업을 말합니다. 
- setLevel('ERROR')는 로깅 레벨을 'ERROR'로 설정하는 것을 의미합니다. 
- 이는 로그 메시지 중에서 오류에 관련된 메시지만 표시하고, 경고 및 정보 메시지는 표시하지 않도록 설정하는 것을 의미합니다. 

In [3]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

84125825/84125825 [==============================] - 133s 2us/step


- tf.keras.utils.get_file 함수는 파일을 다운로드하고 로컬 파일 시스템에 저장하는 유틸리티 함수입니다.
-   tf.keras.utils.get_file(
        fname,
        origin,
        untar=False,
        md5_hash=None,
        file_hash=None,
        cache_dir=None,
        cache_subdir='datasets',
        extract=False,
        archive_format='auto',
        cache_subdir_lock=True
    )
    - fname: 다운로드할 파일의 이름을 지정합니다.
    - origin: 파일의 원격 경로 또는 URL을 지정합니다. url
    - untar (기본값: False): True로 설정하면 tar 파일을 해제합니다. (압축 파일 해제)
    - cache_dir: 다운로드한 파일의 저장 위치를 지정합니다.
    - cache_subdir (기본값: 'datasets'): cache_dir에서 파일을 저장할 하위 디렉토리를 지정합니다.

- dataset_dir: dataset의 디렉토리 경로를 가져온 후, 'aclImdb'라는 폴더명을 추가하여 데이터셋의 최종 저장 위치를 지정합니다.
- train_dir: dataset_dir에서 'train' 폴더의 경로를 생성합니다.
- remove_dir: train_dir에서 'unsup' 폴더의 경로를 생성합니다.
- shutil.rmtree(remove_dir): remove_dir에 해당하는 폴더를 완전히 삭제합니다.
    - unsup 폴더는 IMDB 데이터셋의 일부인 unlabeled 데이터를 포함하는 폴더입니다. 이 폴더에는 레이블이 지정되지 않은 영화 리뷰 데이터가 포함되어 있습니다.
    - 일반적으로 IMDB 데이터셋은 감정 분석을 위해 사용되며, 각 리뷰에는 긍정적인 감정 또는 부정적인 감정이 할당됩니다. 하지만 unsup 폴더의 데이터는 레이블이 없기 때문에 이 데이터는 감정 분석 모델의 비지도 학습에 사용되거나, 다른 목적으로 활용될 수 있습니다.
    - 이 작업은 해당 데이터를 사용하지 않는 경우, 데이터를 삭제하여 불필요한 처리나 메모리 사용을 줄일 수 있도록 합니다.


In [4]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


- AUTOTUNE = tf.data.AUTOTUNE
    - TensorFlow의 AUTOTUNE 기능을 사용합니다. 이는 TensorFlow가 자동으로 최적화된 값을 선택하도록 합니다.

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)
- raw_train_ds: text_dataset_from_directory 
    - 함수를 사용하여 훈련 데이터를 로드합니다. 지정된 디렉토리('aclImdb/train')에서 텍스트 파일을 읽어와서 배치 형태로 데이터셋을 생성합니다. validation_split 인자를 사용하여 훈련 데이터 중 일부를 검증 데이터로 분할하고, subset 인자를 사용하여 훈련 데이터로 선택된 부분을 지정합니다.
    - aclImdb/train : 데이터가 저장된 디렉토리 경로
    - validation_split : 데이터를 훈련 및 검증 데이터로 분할하는 비율을 지정합니다.
    - subset='training' : 'training' 또는 'validation' 중 하나를 선택하여 훈련 데이터 또는 검증 데이터로 지정합니다.

- class_names: 데이터셋에서 사용되는 클래스(레이블)의 이름을 저장합니다.

train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
- train_ds: 훈련 데이터셋을 캐시하고(데이터를 메모리나 디스크에 저장하여 나중에 재사용할 수 있도록 합니다), 학습 전에 필요한 데이터를 미리 로드하여 성능을 향상시킵니다. prefetch 함수를 사용하여 데이터를 비동기적으로 로드합니다.(데이터를 배치 단위로 로드하는 동안 모델의 학습이나 추론 작업을 동시에 수행할 수 있도록 해줍니다.)
    - buffer_size 매개변수는 데이터를 미리 로드해놓을 버퍼의 크기를 지정합니다. 
       - buffer_size=1000으로 설정하면, 다음 배치를 로드하기 전에 1000개의 데이터를 미리 로드해놓습니다.
       - 이렇게 미리 로드해놓은 데이터는 모델이 학습 중에 사용되므로 데이터 로딩에 따른 대기 시간이 줄어들어 전체 학습 과정이 더욱 효율적으로 이루어질 수 있습니다.
       - buffer_size=AUTOTUNE을 설정하면 TensorFlow가 최적의 버퍼 크기를 자동으로 결정하게 됩니다.




